In [13]:
!pip install dash
!pip install dash-bootstrap-components

In [14]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

In [15]:
# 0. Preparar el entorno
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
## 1. Cargar el DataFrame
df = pd.read_excel("/content/drive/MyDrive/CONSOLIDADO DATOS.xlsx",header=0)
df

,NOMBRES Y APELLIDOS,EDAD,CIUDAD,SALARIO,ESTADO CIVIL,GENERO,NÚM. HIJOS,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Alba Ruby Jiménez Bacca,35,Medellín,689700,Casada,F,3,NaN,NaN,NaN
1,Amalia Duque Ramirez,27,Medellín,689700,Soltera,F,0,NaN,NaN,NaN
2,Ana Maria Rincon Delgado,30,Bogotá,689700,Soltero,F,2,NaN,NaN,NaN
3,Ana Milvia Montoya Salazar,30,Bogotá,689700,Soltera,F,2,NaN,NaN,NaN
4,Betsy Arango López,34,Bogotá,689700,Soltera,F,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
237,Ana Meneses Osorio,31,Bogotá,6300000,Casada,F,3,NaN,NaN,NaN
238,Cesar Espitia Barbosa,30,Medellín,6300000,Casado,M,3,NaN,NaN,NaN
239,Stefany Carolina Garzon Rojas,31,Medellín,6300000,Casada,F,3,NaN,NaN,NaN
240,Jenny Barbosa Godoy,25,Bogotá,8400000,Soltera,F,2,NaN,NaN,NaN


## Utilizar http://dash-bootstrap-components.com/. Es una libreria con componentes predefinidos para construir el reporte de una manera mucho más sencilla

In [23]:
## 3. Crear la aplicación Dash
external_stylesheets = [dbc.themes.QUARTZ]  ## Podemos usar diferentes temas preconfigurados
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Reporte de pedidos de comida online"

https://www.dash-bootstrap-components.com/docs/themes/  Temas que puedo usar

In [36]:
### 4. Definir el layout de la aplicación
# Definir las opciones y rangos antes de construir el layout
estado_civil_unique = df['ESTADO CIVIL'].unique()
genero_unique = df['GENERO'].unique()
edad_min_val = df['EDAD'].min()
edad_max_val = df['EDAD'].max()
numero_hijos_unique = df['NÚM. HIJOS'].unique()

app.layout = dbc.Container([
    dbc.Row([html.H1("Reporte de pedidos de comida online")]),
    dbc.Row([
          dbc.Col([
              html.Label("Selecciona el estado civil:"), # Corrected label
              dcc.Dropdown(
              id='selector_estado_civil',
              options=[{'label': ec, 'value': ec} for ec in estado_civil_unique], # Corrected loop
              value=estado_civil_unique[0]  # Valor por defecto
          )]),
          dbc.Col([
              html.Label("Selecciona el género:"), # Corrected label
              dcc.RadioItems(
              id='selector_genero',
              options=[{'label': g, 'value': g} for g in genero_unique], # Corrected loop
              value=genero_unique[0]  # Valor por defecto
          )]),
          dbc.Col([
              html.Label("Selecciona el rango de edad:"),
              dcc.RangeSlider(
              id='selector_edad',
              min=edad_min_val, # Using defined variable
              max=edad_max_val, # Using defined variable
              value=[edad_min_val, edad_max_val],
              marks={i: str(i) for i in range(edad_min_val, edad_max_val + 1)}, # Corrected 'edad' function
              step=1
          )]),
          dbc.Col([
              html.Label("Selecciona el número de hijos:"), # Corrected label to match column
              dbc.Checklist(
                  id='selector_numero_hijos',
                  options=[{'label': num_hijo, 'value': num_hijo} for num_hijo in numero_hijos_unique], # Corrected loop
                  value=[numero_hijos_unique[0]],
              )
          ])

        ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='grafica_barras')
        ]),
        dbc.Col([
            dcc.Graph(id='grafica_torta')
        ]),
        dbc.Col([
            dcc.Graph(id='grafica_area')
        ])
    ])
])

In [38]:
### 5. Definir los callbacks (Si es necesario)
@app.callback(
    [Output('grafica_barras', 'figure'),
     Output('grafica_torta', 'figure'),
     Output('grafica_area', 'figure')],
    [Input('selector_ocupacion', 'value'),
     Input('selector_genero', 'value'),
     Input('selector_edad', 'value'),
     Input('selector_feedback', 'value'),]
)

def crear_graficas(valor_ocupacion, valor_genero, valor_edad, valor_feedback):
    # Filtrar el DataFrame según el género seleccionado
    if valor_genero is None:
      df_filtrado = df[(df['ESTADO CIVIL'] == estado_civil_unique) & (df['EDAD'] >= valor_edad[0]) & (df['EDAD'] <= valor_edad[1]) & (df['NÚM. HIJOS'].isin(numero_hijos_unique))]
    else:
       df_filtrado = df[(df['ESTADO CIVIL'] == estado_civil_unique) & (df['EDAD'] == valor_edad) & (df['EDAD'] >= valor_edad[0]) & (df['EDAD'] <= valor_edad[1]) & (df['NÚM. HIJOS'].isin(numero_hijos_unique))]


    conteo_feedback_estado_civil = df_filtrado.groupby(['Marital Status','NÚM. HIJOS'])['EDAD'].count().reset_index().sort_values(by='NÚM. HIJOS', ascending=False)

    grafica_barras = px.bar(conteo_feedback_estado_civil,
                            x='Marital Status',
                            y='EDAD',
                            color = 'NÚM. HIJOS',
                            title='Promedio del tamaño de la familia por estado civil',
                            color_discrete_sequence=['#65c78c', '#f74a50'])

    conteo_votos_feedback = df_filtrado.groupby(['NÚM. HIJOS'])['EDAD'].count().reset_index().sort_values(by='NÚM. HIJOS', ascending=False)
    grafica_torta = px.pie(conteo_votos_feedback,
                            names='NÚM HIJOS',
                            values='EDAD',
                            title='Distribución del tamaño de la familia por estado civil',
                            color_discrete_sequence=['#65c78c', '#f74a50'])

    grafica_area = px.area(df_filtrado, x="Educational Qualifications", y="Family size", color="GENERO",title='Gráfica de area',)


    ## Esto me sirve para que el fondo de las gráficas sea transparente
    grafica_barras.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })

    grafica_torta.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })

    grafica_area.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })

    return grafica_barras, grafica_torta, grafica_area


In [ ]:
if __name__ == '__main__':
    app.run()

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8050
INFO:werkzeug:Press CTRL+C to quit
